In [17]:
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-' + level + '.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'
file_name_icu_r = path_csv + '/terapie-intensive.csv'

In [18]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 28),
 datetime.date(2020, 10, 27),
 datetime.date(2020, 10, 26))

In [19]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201028.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201027.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201026.csv')

In [20]:
if False:
    while True:
        status_code = requests.get(url_tday).status_code
        if status_code == 200:
            print('Presenti dati odierni')
            break
        print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)

In [21]:
# Get csv files
try:
    # today and yesterday if today present
    df_raw_r_1 = pd.read_csv(url_tday)
    print('Dati odierni')
    df_raw_r_0 = pd.read_csv(url_yday)
    date_last = date_tday
except Exception as e:
    # yesterday and day before yesterday otherwise
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)
    date_last = date_yday
    print('Dati di ieri')

Dati odierni


In [22]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-28T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2237,135,2372,23415,...,2547,2827,32353,4305,30854,31591,62445,984720,599224,NaN
1,2020-10-28T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,107,7,114,1316,...,63,77,1216,157,2496,307,2803,39314,24935,NaN
2,2020-10-28T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3072,292,3364,58042,...,6522,7558,91763,17357,138923,31603,170526,2808259,1755084,NaN
3,2020-10-28T17:00:00,ITA,5,Veneto,45.434905,12.338452,708,90,798,20802,...,2083,2143,25180,2355,24377,24758,49135,2275869,886165,NaN
4,2020-10-28T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,146,35,181,3551,...,287,406,5028,382,8208,934,9142,521139,223790,NaN
5,2020-10-28T17:00:00,ITA,7,Liguria,44.411493,8.932699,878,46,924,6554,...,521,926,16312,1733,18076,7447,25523,424986,221286,NaN
6,2020-10-28T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,999,119,1118,17112,...,1150,1212,27671,4593,38677,11817,50494,1531311,827354,In seguito a verifica sui dati comunicati nei ...
7,2020-10-28T17:00:00,ITA,9,Toscana,43.769231,11.255889,857,130,987,21373,...,1320,1708,13335,1297,29543,7449,36992,1041270,696653,NaN
8,2020-10-28T17:00:00,ITA,10,Umbria,43.106758,12.388247,255,37,292,4707,...,238,401,3160,114,2723,5550,8273,286380,167083,NaN
9,2020-10-28T17:00:00,ITA,11,Marche,43.616760,13.518875,250,32,282,4244,...,301,351,6875,1008,12409,0,12409,310091,181572,NaN


In [23]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-27T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2016,121,2137,21103,...,1973,2458,32092,4286,30000,29618,59618,968047,585298,NaN
1,2020-10-27T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,99,7,106,1261,...,51,57,1203,156,2428,298,2726,38828,24643,NaN
2,2020-10-27T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2715,271,2986,51898,...,4559,5035,90774,17310,133215,29753,162968,2766999,1729725,NaN
3,2020-10-27T17:00:00,ITA,5,Veneto,45.434905,12.338452,664,83,747,18770,...,1470,1526,25131,2344,24220,22772,46992,2259786,880224,NaN
4,2020-10-27T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,139,31,170,3275,...,160,241,4913,378,7886,850,8736,515566,222085,NaN
5,2020-10-27T17:00:00,ITA,7,Liguria,44.411493,8.932699,837,45,882,6075,...,515,1127,15922,1718,17732,6865,24597,418646,218449,NaN
6,2020-10-27T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,934,103,1037,16043,...,1311,1413,27626,4579,38049,11236,49285,1509935,818615,In seguito a verifica sui dati comunicati nei ...
7,2020-10-27T17:00:00,ITA,9,Toscana,43.769231,11.255889,804,119,923,20117,...,1664,1823,12954,1290,28054,7230,35284,1027459,688497,effettuati 7134 tamponi rapidi antigenici di c...
8,2020-10-27T17:00:00,ITA,10,Umbria,43.106758,12.388247,248,37,285,4476,...,178,314,3001,110,2672,5200,7872,281634,164539,NaN
9,2020-10-27T17:00:00,ITA,11,Marche,43.616760,13.518875,250,28,278,3947,...,303,380,6827,1006,12058,0,12058,307045,179841,NaN


In [24]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-28T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2237,135,2372,23415,...,2547,2827,32353,4305,30854,31591,62445,984720,599224,NaN
1,2020-10-28T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,107,7,114,1316,...,63,77,1216,157,2496,307,2803,39314,24935,NaN
2,2020-10-28T17:00:00,ITA,3,Lombardia,45.466794,9.190347,3072,292,3364,58042,...,6522,7558,91763,17357,138923,31603,170526,2808259,1755084,NaN
3,2020-10-28T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,275,21,296,5392,...,302,381,9116,736,11931,3609,15540,505278,228843,NaN
4,2020-10-28T17:00:00,ITA,5,Veneto,45.434905,12.338452,708,90,798,20802,...,2083,2143,25180,2355,24377,24758,49135,2275869,886165,NaN
5,2020-10-28T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,146,35,181,3551,...,287,406,5028,382,8208,934,9142,521139,223790,NaN
6,2020-10-28T17:00:00,ITA,7,Liguria,44.411493,8.932699,878,46,924,6554,...,521,926,16312,1733,18076,7447,25523,424986,221286,NaN
7,2020-10-28T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,999,119,1118,17112,...,1150,1212,27671,4593,38677,11817,50494,1531311,827354,In seguito a verifica sui dati comunicati nei ...
8,2020-10-28T17:00:00,ITA,9,Toscana,43.769231,11.255889,857,130,987,21373,...,1320,1708,13335,1297,29543,7449,36992,1041270,696653,NaN
9,2020-10-28T17:00:00,ITA,10,Umbria,43.106758,12.388247,255,37,292,4707,...,238,401,3160,114,2723,5550,8273,286380,167083,NaN


In [25]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-27T17:00:00,ITA,1,Piemonte,45.073274,7.680687,2016,121,2137,...,1973,2458,32092,4286,30000,29618,59618,968047,585298,NaN
1,17,2020-10-27T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,99,7,106,...,51,57,1203,156,2428,298,2726,38828,24643,NaN
2,8,2020-10-27T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2715,271,2986,...,4559,5035,90774,17310,133215,29753,162968,2766999,1729725,NaN
3,19,2020-10-27T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,253,17,270,...,445,499,9046,727,11613,3546,15159,500428,227549,NaN
4,18,2020-10-27T17:00:00,ITA,5,Veneto,45.434905,12.338452,664,83,747,...,1470,1526,25131,2344,24220,22772,46992,2259786,880224,NaN
5,5,2020-10-27T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,139,31,170,...,160,241,4913,378,7886,850,8736,515566,222085,NaN
6,7,2020-10-27T17:00:00,ITA,7,Liguria,44.411493,8.932699,837,45,882,...,515,1127,15922,1718,17732,6865,24597,418646,218449,NaN
7,4,2020-10-27T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,934,103,1037,...,1311,1413,27626,4579,38049,11236,49285,1509935,818615,In seguito a verifica sui dati comunicati nei ...
8,15,2020-10-27T17:00:00,ITA,9,Toscana,43.769231,11.255889,804,119,923,...,1664,1823,12954,1290,28054,7230,35284,1027459,688497,effettuati 7134 tamponi rapidi antigenici di c...
9,16,2020-10-27T17:00:00,ITA,10,Umbria,43.106758,12.388247,248,37,285,...,178,314,3001,110,2672,5200,7872,281634,164539,NaN


In [26]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [27]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [28]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-28T17:00:00,1,Piemonte,2237,135,2372,23415,25787,2547,2827,32353,4305,30854,31591,62445,984720,599224,4341375
1,2020-10-28T17:00:00,2,Valle d'Aosta,107,7,114,1316,1430,63,77,1216,157,2496,307,2803,39314,24935,125501
2,2020-10-28T17:00:00,3,Lombardia,3072,292,3364,58042,61406,6522,7558,91763,17357,138923,31603,170526,2808259,1755084,10103969
3,2020-10-28T17:00:00,4,Trentino-Alto Adige,275,21,296,5392,5688,302,381,9116,736,11931,3609,15540,505278,228843,1074819
4,2020-10-28T17:00:00,5,Veneto,708,90,798,20802,21600,2083,2143,25180,2355,24377,24758,49135,2275869,886165,4907704
5,2020-10-28T17:00:00,6,Friuli Venezia Giulia,146,35,181,3551,3732,287,406,5028,382,8208,934,9142,521139,223790,1211357
6,2020-10-28T17:00:00,7,Liguria,878,46,924,6554,7478,521,926,16312,1733,18076,7447,25523,424986,221286,1543127
7,2020-10-28T17:00:00,8,Emilia-Romagna,999,119,1118,17112,18230,1150,1212,27671,4593,38677,11817,50494,1531311,827354,4467118
8,2020-10-28T17:00:00,9,Toscana,857,130,987,21373,22360,1320,1708,13335,1297,29543,7449,36992,1041270,696653,3722729
9,2020-10-28T17:00:00,10,Umbria,255,37,292,4707,4999,238,401,3160,114,2723,5550,8273,286380,167083,880285


In [29]:
# Merge with icu data
df_icu_r = pd.read_csv(file_name_icu_r)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-10-28T17:00:00,1,Piemonte,2237,135,2372,23415,25787,2547,2827,32353,4305,30854,31591,62445,984720,599224,4341375,367
1,2020-10-28T17:00:00,2,Valle d'Aosta,107,7,114,1316,1430,63,77,1216,157,2496,307,2803,39314,24935,125501,20
2,2020-10-28T17:00:00,3,Lombardia,3072,292,3364,58042,61406,6522,7558,91763,17357,138923,31603,170526,2808259,1755084,10103969,983
3,2020-10-28T17:00:00,4,Trentino-Alto Adige,275,21,296,5392,5688,302,381,9116,736,11931,3609,15540,505278,228843,1074819,106
4,2020-10-28T17:00:00,5,Veneto,708,90,798,20802,21600,2083,2143,25180,2355,24377,24758,49135,2275869,886165,4907704,825
5,2020-10-28T17:00:00,6,Friuli Venezia Giulia,146,35,181,3551,3732,287,406,5028,382,8208,934,9142,521139,223790,1211357,175
6,2020-10-28T17:00:00,7,Liguria,878,46,924,6554,7478,521,926,16312,1733,18076,7447,25523,424986,221286,1543127,209
7,2020-10-28T17:00:00,8,Emilia-Romagna,999,119,1118,17112,18230,1150,1212,27671,4593,38677,11817,50494,1531311,827354,4467118,516
8,2020-10-28T17:00:00,9,Toscana,857,130,987,21373,22360,1320,1708,13335,1297,29543,7449,36992,1041270,696653,3722729,415
9,2020-10-28T17:00:00,10,Umbria,255,37,292,4707,4999,238,401,3160,114,2723,5550,8273,286380,167083,880285,70


In [30]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [31]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-28T17:00:00,3,Lombardia,3072,292,3364,58042,61406,6522,7558,...,29.70,6144,2523,41260,25359,47,989,18.32,0.07480,1.69
1,2020-10-28T17:00:00,1,Piemonte,2237,135,2372,23415,25787,2547,2827,...,36.78,2312,369,16673,13926,19,261,16.96,0.06512,1.44
2,2020-10-28T17:00:00,15,Campania,1235,160,1395,33491,34886,2045,2427,...,37.47,2000,-334,15030,8965,17,365,16.15,0.04195,0.79
3,2020-10-28T17:00:00,5,Veneto,708,90,798,20802,21600,2083,2143,...,10.91,2032,617,16083,5941,11,49,13.32,0.04367,1.00
4,2020-10-28T17:00:00,12,Lazio,1669,166,1835,26111,27946,1804,1963,...,22.22,1767,-30,22947,19948,19,140,8.55,0.03347,0.68
5,2020-10-28T17:00:00,9,Toscana,857,130,987,21373,22360,1320,1708,...,31.33,1256,-115,13811,8156,7,381,12.37,0.04588,0.99
6,2020-10-28T17:00:00,8,Emilia-Romagna,999,119,1118,17112,18230,1150,1212,...,23.06,1069,-201,21376,8739,14,45,5.67,0.02713,1.13
7,2020-10-28T17:00:00,7,Liguria,878,46,924,6554,7478,521,926,...,22.01,479,-201,6340,2837,15,390,14.61,0.06001,1.65
8,2020-10-28T17:00:00,16,Puglia,690,70,760,8677,9437,729,772,...,19.13,666,161,6437,5934,13,30,11.99,0.01926,0.41
9,2020-10-28T17:00:00,19,Sicilia,787,111,898,11290,12188,454,708,...,20.63,386,-152,7499,4342,10,244,9.44,0.01425,0.38


In [32]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)